# Декораторы функций

Механизм декораторов напоминает аннотации в языке C#. Но в C# аннотация являлась пассивной структурой, которая  "приклеивалась" к необходимому элементу и была доступна в откомпилированной сборке.

Декоратор функции представляет собой функцию высшего порядка, и, в отличие от аннотации, выполняет необходимый код.

Работа декораторов строится на том, что функция в Питоне является "объектом первого порядка" - ее можно вернуть из функции или передать в функцию.

In [1]:
def return_func():

    def func_inside():
        print("I'm inside")

    print("I'm outside")
    return func_inside

In [2]:
return_func()

I'm outside


<function __main__.return_func.<locals>.func_inside()>

In [3]:
return_func()()

I'm outside
I'm inside


In [4]:
def accept_func(some_func):
    print("accept_func")
    some_func()


def my_func():
    print("my_func")


accept_func(my_func)

accept_func
my_func


In [5]:
def decorator(some_func):
    print("before")
    some_func()
    print("after")


def my_func():
    print("my_func")


decorator(my_func)

before
my_func
after


In [6]:
def my_decorator(func_to_decorate): # 1. Функция, которая

    def decorated_func():
        print("before")
        func_to_decorate() # 3. В которую завернута декорируемая функция
        print("after")

    return decorated_func # 2. возвращает функцию,


def my_func():
    print("my_func")


decorated = my_decorator(my_func)
decorated()

before
my_func
after


In [7]:
# Синтаксис для обертывания функции в декоратор
@my_decorator
def my_func2():
    print("my_func2")

In [8]:
my_func2()

before
my_func2
after


In [9]:
def decorator_creator(decorator_arg):
    def my_decorator(func_to_decorate):
        def decorated_func():
            # Мы обращаемся к параметрам с использованием 
            # механизма замыканий, здесь это оправдано
            print("decorator_arg = {}".format(decorator_arg))
            func_to_decorate()
        return decorated_func
    return my_decorator


@decorator_creator(5) # Результат вызова - my_decorator
def my_func3():
    print("my_func3")

my_func3()

decorator_arg = 5
my_func3


In [10]:
#my_func3(1,2,3)

In [11]:
def benchmark(func):
    import time

    def wrapper(*args, **kwargs):
        t = time.time()
        res = func(*args, **kwargs)
        print(func.__name__, time.time() - t)
        return res

    return wrapper

In [12]:
def logging(func):
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print(func.__name__, args, kwargs)
        return res

    return wrapper

In [13]:
@logging
def my_func4(p1, p2):
    print(p1, p2)

In [14]:
my_func4(1,2)

1 2
my_func4 (1, 2) {}


In [15]:
def logging_with_params(par1, par2):
    def logging(func):
        def wrapper(*args, **kwargs):
            print('Decorator arguments:', par1, par2)
            res = func(*args, **kwargs)
            print(func.__name__, args, kwargs)
            return res

        return wrapper
    return logging

In [16]:
@logging_with_params(1,2)
def my_func5(p1, p2):
    print(p1, p2)

In [17]:
my_func5(1,2)

Decorator arguments: 1 2
1 2
my_func5 (1, 2) {}


In [18]:
def logging_with_params2(*args, **kwargs):
    
    print('=====================================')
    print('Arguments:')
    for arg in args:
        print(arg)
    print('\nKeywords:')
    for key in kwargs:
        print('{} = {}'.format(key, kwargs[key]))       
    print('=====================================')
    
    par1 = args[0]
    par2 = args[1]
    
    def logging(func):
        def wrapper(*args, **kwargs):
            print('Decorator arguments:', par1, par2)
            res = func(*args, **kwargs)
            print(func.__name__, args, kwargs)
            return res

        return wrapper
    return logging

In [19]:
@logging_with_params2(1,2,3,True, 'qwerty', param1=123, bool_param=True, str_param='string')
def my_func7(p1, p2):
    print(p1, p2)

Arguments:
1
2
3
True
qwerty

Keywords:
param1 = 123
bool_param = True
str_param = string


In [20]:
my_func7(1,2)

Decorator arguments: 1 2
1 2
my_func7 (1, 2) {}


In [21]:
@benchmark
def time1(n):
    res = []
    for i in range(n):
        res.append(i)
    return res

In [22]:
tt1 = time1(10000000)

time1 0.8724780082702637


In [23]:
@benchmark
def time2(n):
    return [i for i in range(n)]

In [24]:
tt2 = time2(10000000)

time2 0.4886922836303711


## Использование нескольких декораторов

In [25]:
def add_decorator(param):
    def inner(func):
        def wrapper(*args, **kwargs):
            print('add_decorator:', param)
            res = func(*args, **kwargs)
            return res + param
        return wrapper
    return inner

In [26]:
def mul_decorator(param):
    def inner(func):
        def wrapper(*args, **kwargs):
            print('mul_decorator:', param)
            res = func(*args, **kwargs)
            return res * param
        return wrapper
    return inner

In [27]:
def pow_decorator(param):
    def inner(func):
        def wrapper(*args, **kwargs):
            print('pow_decorator:', param)
            res = func(*args, **kwargs)
            return res ** param
        return wrapper
    return inner

In [28]:
@mul_decorator(5)
@add_decorator(1)
def get_const_1(i):
    return i

In [29]:
get_const_1(3)

mul_decorator: 5
add_decorator: 1


20

In [30]:
@pow_decorator(2)
@mul_decorator(5)
@add_decorator(1)
def get_const_2(i):
    return i

In [31]:
get_const_2(3)

pow_decorator: 2
mul_decorator: 5
add_decorator: 1


400

**Вывод:** декораторы выполняются от внутреннего к внешнему.